In [1]:
import re
import pandas as pd

In [2]:
results_output = open('data/mnpoll_201910_raw.txt', 'r').read()

In [3]:
question_finder = r'(Task number.+?)\*+\n\*+'
questions = re.findall(question_finder, results_output, re.DOTALL)

In [4]:
def parse_question_answers(q_text):
    '''Potential answers are 1 or 2 lines of text between a row of ---- and and row of ==='''
    q_answers_raw = re.search(r'-+\s+\n(.+?)\n\s+=', q_text, re.DOTALL).group(1)
    q_answers_lines = q_answers_raw.split('\n')
    q_answers_lines_parsed = []
    for line in q_answers_lines:
        # Find the word groups in each row. Right now this only works if the label is one word, no spaces. But could be modified
        q_answers_lines_parsed.append(re.findall(r'\s([\w\d]+)', line))
    
    first_row = q_answers_lines_parsed[0]
    for row in q_answers_lines_parsed[1:]:
        for key, cell in enumerate(row):
            first_row[key] += cell
        
    return first_row

In [5]:
def parse_table(table_text, choices):
    # Lines in a table body have two line breaks between them
    lines = re.split(r'\n\s+\n', table_text)
    table_data = []
    for l in lines:
        if l != '':
            line_label = re.search(r'(.+?)\s{2,}', l).group(1)
            line_data = re.findall(r'\s{2,}([\d\.]+)', l)
            line_data_tagged = [{'choice': choices[k], 'value': cell} for k, cell in enumerate(line_data)]
            table_data.append({
                'segment': line_label,
                'data': line_data_tagged
            })
    return table_data

In [6]:
def parse_segments(q_text, q_choices):
    # Find segment headers and content of results table until next tilde or end of string
    segment_finder = r'~([\w\s]+) \n-+(.+?(?=~|\Z))'
    segments = re.findall(segment_finder, q_text, re.DOTALL)
    segment_list = []
    for s in segments:
        segment_name = 'OVERALL' if s[0] == 'RESULTS' else s[0]
        segment_list.append({
            'segment_name': segment_name,
            'table_data': parse_table(s[1], q_choices)
        })
    return segment_list

In [7]:
BREAKDOWN_ORDER = ['OVERALL', 'SEX', 'PARTY ID', 'AGE', 'INCOME', 'REGION']
SEGMENT_REPLACE_LOOKUP = {
    'STATE': 'OVERALL',
    'MALE': 'Men',
    'FEMALE': 'Women',
    'DEM/DFL': 'DFL / Democrat',
    'REP': 'Republican',
    'IND': 'Independent / Other',
    '18-34': '18-34',
    '35-49': '35-49',
    '50-64': '50-64',
    '65+': '65+',
    'REF': 'REF',
    '<$50,000': 'Under $50,000',
    '$50,000+': '$50,000 and over',
    'HENNEPIN/RAMSEY': 'Hennepin / Ramsey',
    'METRO SUBURBS': 'Metro Suburbs',
    'SOUTHERN MN': 'Southern Minn.',
    'NORTHERN MN': 'Northern Minn.',
}
SEGMENT_ORDER = list(SEGMENT_REPLACE_LOOKUP.keys())

In [8]:
def make_dataframe(q_title, q_choices, q_segments):
    df = pd.DataFrame()
    for segment_type in q_segments:
        segment_name = segment_type['segment_name']
        for segment in segment_type['table_data']:
            seg_df = pd.DataFrame(segment['data'])
            seg_df['value'] = seg_df['value'].astype(float)
            seg_df['segment'] = segment['segment']
            seg_df['breakdown'] = segment_name
            df = df.append(seg_df)

    # Pivot to video, er, Strib style
    pt = df.pivot_table('value', ['breakdown', 'segment'], 'choice').reset_index()

    # Change order
    def assign_order(value, ordering_keys):
        return ordering_keys[value]
    breakdown_ordering_keys = {v: k for k, v in enumerate(BREAKDOWN_ORDER)}
    segment_ordering_keys = {v: k for k, v in enumerate(SEGMENT_ORDER)}
    pt['breakdown_order'] = pt.breakdown.apply(assign_order, args=(breakdown_ordering_keys,))
    pt['segment_order'] = pt.segment.apply(assign_order, args=(segment_ordering_keys,))
    pt.sort_values(['breakdown_order', 'segment_order'], inplace=True)
    
    # Make Strib style
    def reformat_segment(value):
        return SEGMENT_REPLACE_LOOKUP[value]
    pt['segment'] = pt.segment.map(reformat_segment)
    
    # Drop temp columns
    pt.drop(columns=['breakdown_order', 'segment_order'], inplace=True)
    
    # Add "raw" columns and round orig columns
    for c in q_choices:
        pt['{}_raw'.format(c)] = pt[c]
        pt[c] = pt[c].astype("float").round()
    return pt

In [9]:
def format_value(str_input, add_pct):
    pct_value = '%' if add_pct else ''
    return '{}{}'.format(str(round(float(str_input))), pct_value)

In [10]:
def build_topline_chart(segments):
    html = ''
    chart_data = segments['table_data'][0]['data']
    header_items = ['<td class="label" width="{}">{}</td>'.format(format_value(dp['value'], True), dp['choice']) for dp in chart_data]
    header = '<tr>{}</tr>'.format(''.join(header_items))
    
    body_items = ['<td class="bar" width="{}">{}</td>'.format(format_value(dp['value'], True), format_value(dp['value'], True)) for dp in chart_data]
    body = '<tr>{}</tr>'.format(''.join(body_items))
    return '\n<table class="stacked-bar-graph"><tbody>\n{}\n{}\n</tbody></table>\n'.format(header, body)

In [11]:
def build_topline_chart_df(q_choices, df):
    html = ''

    header_items = ['<td class="label" width="{}">{}</td>\n'.format(df[choice], choice.title()) for choice in q_choices]
    header = '<tr>\n{}</tr>\n'.format(''.join(header_items))
    
    body_items = ['<td class="bar" width="{}">{}</td>\n'.format(format_value(df[choice], True), format_value(df[choice], True)) for choice in q_choices]
    body = '<tr>{}</tr>'.format(''.join(body_items))
    return '\n<table class="stacked-bar-graph"><tbody>\n{}\n{}\n</tbody></table>\n'.format(header, body)

In [12]:
def build_question_html_df(q_title, q_choices, q_df):
    h_df = q_df.copy()

    # Build topline (first row) into chart
    topline_df = h_df.iloc[0]
    topline_chart = build_topline_chart_df(q_choices, topline_df)

    # Remove first row since it's in chart
    h_df = h_df.drop(h_df.index[0]).reset_index().drop(columns=["index"])
    
    # Remove REF row
    h_df = h_df[h_df['segment'] != 'REF']
    
    # Drop raw columns
    h_df = h_df[h_df.columns.drop(list(h_df.filter(regex='_raw')))]
    
    # Apply % to new first row
    def add_pct(row):
        if row.name == 0:
            return row.apply(lambda x: '{}%'.format(str(int(x))))
        return row.apply(lambda x: str(int(x)))
    h_df[q_choices] = h_df[q_choices].apply(add_pct, axis=1)
    
    html = '<thead>\n<tr><th>&nbsp;</th><th>{}</th></tr>\n</thead>\n'.format('</th><th>'.join(q_choices))
    for breakdown in BREAKDOWN_ORDER:
        if breakdown not in 'OVERALL':
            for index, row in h_df[h_df['breakdown'] == breakdown].copy().reset_index().iterrows():
                choice_values = [str(row[x]) for x in q_choices]
                if index == h_df[h_df['breakdown'] == breakdown].shape[0] - 1:
                    tr_class = ' class="last_line"'
                else:
                    tr_class = ''
                html += '<tr{}><th scope="row">{}</th><td>{}</td></tr>\n'.format(tr_class, row['segment'], '</td><td>'.join(choice_values))

    return '<h2>{}</h2>\n{}<table class="poll-question">\n<tbody>\n{}</tbody>\n</table>\n'.format(q_title, topline_chart, html)

In [13]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('data/poll_findings.xlsx', engine='xlsxwriter')

for k, q in enumerate(questions):
    # Question title is first line that starts with a tilde
    q_title = re.search(r'~([\w\d\s\?\-]+?)\n', q).group(1).strip()
    q_choices = parse_question_answers(q)
    q_segments = parse_segments(q, q_choices)
    
    # Make into sensible pandas dataframe
    q_df = make_dataframe(q_title, q_choices, q_segments)
#     display(q_df)

    # Clean up q title for Excel
    worksheet_title = re.sub(r'[\?]', '', '{}) {}'.format(k, q_title))
    # Write each dataframe to a different worksheet.
    q_df.to_excel(writer, sheet_name=worksheet_title, index=False)
    
#     q_html = build_question_html(q_title, q_choices, q_segments)
    q_html_df = build_question_html_df(q_title, q_choices, q_df)
    print(q_html_df)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

<h2>PRES VOTE 1</h2>

<table class="stacked-bar-graph"><tbody>
<tr>
<td class="label" width="50.0">Biden</td>
<td class="label" width="38.0">Trump</td>
<td class="label" width="12.0">Und</td>
</tr>

<tr><td class="bar" width="50%">50%</td>
<td class="bar" width="38%">38%</td>
<td class="bar" width="12%">12%</td>
</tr>
</tbody></table>
<table class="poll-question">
<tbody>
<thead>
<tr><th>&nbsp;</th><th>BIDEN</th><th>TRUMP</th><th>UND</th></tr>
</thead>
<tr><th scope="row">Men</th><td>46%</td><td>41%</td><td>13%</td></tr>
<tr class="last_line"><th scope="row">Women</th><td>54</td><td>35</td><td>11</td></tr>
<tr><th scope="row">DFL / Democrat</th><td>87</td><td>2</td><td>11</td></tr>
<tr><th scope="row">Republican</th><td>12</td><td>80</td><td>8</td></tr>
<tr class="last_line"><th scope="row">Independent / Other</th><td>44</td><td>38</td><td>18</td></tr>
<tr><th scope="row">18-34</th><td>61</td><td>26</td><td>13</td></tr>
<tr><th scope="row">35-49</th><td>49</td><td>39</td><td>12</td></t

<h2>IMPEACHMENT INQUIRY</h2>

<table class="stacked-bar-graph"><tbody>
<tr>
<td class="label" width="52.0">Serious</td>
<td class="label" width="43.0">Political</td>
<td class="label" width="5.0">Dk</td>
</tr>

<tr><td class="bar" width="52%">52%</td>
<td class="bar" width="43%">43%</td>
<td class="bar" width="5%">5%</td>
</tr>
</tbody></table>
<table class="poll-question">
<tbody>
<thead>
<tr><th>&nbsp;</th><th>SERIOUS</th><th>POLITICAL</th><th>DK</th></tr>
</thead>
<tr><th scope="row">Men</th><td>48%</td><td>47%</td><td>5%</td></tr>
<tr class="last_line"><th scope="row">Women</th><td>56</td><td>38</td><td>6</td></tr>
<tr><th scope="row">DFL / Democrat</th><td>96</td><td>2</td><td>2</td></tr>
<tr><th scope="row">Republican</th><td>6</td><td>90</td><td>4</td></tr>
<tr class="last_line"><th scope="row">Independent / Other</th><td>46</td><td>43</td><td>11</td></tr>
<tr><th scope="row">18-34</th><td>69</td><td>25</td><td>6</td></tr>
<tr><th scope="row">35-49</th><td>52</td><td>42</td><td>